In [ ]:
import torch
import torch.nn as nn
# Implementa uma rede neural feedforward com duas camadas ocultas e ReLU.

class Neuralnetwork(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = nn.Sequential(
            # Cria a primeira camada oculta linear
            nn.Linear(num_inputs, 30),
            nn.ReLU(),

            # Cria a segunda camada oculta linear
            nn.Linear(30, 50),
            nn.ReLU(),

            # Camada de saída
            nn.Linear(50, num_outputs)
        )

    def forward(self, x):
        return self.layers(x)

SyntaxError: expected ':' (1112629522.py, line 6)